<a href="https://colab.research.google.com/github/gcosma/COP509/blob/main/TutorialRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COP509: Natural Language Processing
# Tutorial: Retrieval-Augmented Generation (RAG)

**Department of Computer Science, Loughborough University**

**Prof. Georgina Cosma** (g.cosma@lboro.ac.uk)

---

## 🎯 Learning Outcomes

By the end of this tutorial, you will be able to:

1. Explain what RAG is and why it addresses key limitations of large language models
2. Implement the retrieval component using vector similarity search
3. Understand how retrieved context is integrated with LLM prompts
4. Build a complete RAG pipeline in Python
5. Evaluate RAG systems and identify common failure modes

---

## 📚 Prerequisites

This tutorial builds on concepts from:
- **Weeks 2-3**: Vector Space Models (TF-IDF, word embeddings)
- **Week 4**: Similarity measures (cosine similarity)
- **Week 5**: LSI and semantic similarity
- **Week 6**: Transformer models (BERT, attention mechanism)

---

## 📖 How to Use This Notebook

| Symbol | Meaning |
|--------|--------|
| 📝 | Important concept - read carefully |
| 💻 | Code to run |
| ✅ | Exercise for you to complete |
| ⚠️ | Common mistake or warning |
| 🔑 | Key takeaway |

**Instructions:**
1. **Run cells in order** - each cell depends on previous cells
2. **Read the explanations** - don't just run the code!
3. **Check expected outputs** - verify your results match
4. **Complete the exercises** - practice is essential for learning

---

## Part 1: Setup and Installation

### 📝 What are we installing?

| Library | Purpose | Why we need it |
|---------|---------|----------------|
| `sentence-transformers` | Creates dense embeddings | Converts text to vectors that capture meaning |
| `chromadb` | Vector database | Stores and searches embeddings efficiently |
| `langchain` | LLM framework | Provides tools to build RAG pipelines |
| `transformers` | Pre-trained models | Gives us access to language models like FLAN-T5 |

**⏱️ Note:** Installation takes 2-3 minutes. You only need to run this once per Colab session.

In [ ]:
# 💻 STEP 1: Install required packages
# The -q flag means 'quiet' - it reduces output clutter
print("📦 Installing packages... this may take 2-3 minutes...")
!pip install -q sentence-transformers chromadb langchain-text-splitters
!pip install -q transformers accelerate
print("✅ Installation complete!")

📦 Installing packages... this may take 2-3 minutes...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.8 MB/s eta 0:00:00


In [ ]:
# 💻 STEP 2: Import libraries
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

**Expected output:**
```
✅ Libraries imported successfully!
```

**⚠️ Troubleshooting:** If you see errors:
- Try running the installation cell again
- Go to **Runtime → Restart runtime**, then run both cells again
- Make sure you're connected to the internet

---

## Part 2: Understanding the Problem

### 📝 What is an LLM?

**LLM** = **L**arge **L**anguage **M**odel

These are neural networks trained on massive amounts of text to understand and generate human language.

**Examples:** ChatGPT, Claude, BERT, GPT-4, FLAN-T5

### 📝 The Problem: Why LLMs Need Help

LLMs have three major limitations:

| Limitation | What it means | Example |
|------------|--------------|--------|
| **Knowledge Cutoff** | Only knows information from training data | Cannot answer "Who won the 2025 election?" |
| **Hallucinations** | Generates plausible but FALSE information | May invent statistics or citations |
| **No Private Data** | Cannot access your documents | Cannot answer "What is our company's leave policy?" |

### 📝 The Solution: RAG

**RAG** = **R**etrieval-**A**ugmented **G**eneration

```
┌─────────────────────────────────────────────────────────────────┐
│                        RAG PIPELINE                             │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   User Query ──────┐                                           │
│                    │                                           │
│                    ▼                                           │
│            ┌──────────────┐      ┌──────────────────┐         │
│            │   RETRIEVE   │ ───► │  Relevant Docs   │         │
│            │   (Search)   │      │  from Database   │         │
│            └──────────────┘      └────────┬─────────┘         │
│                                           │                    │
│                    ┌──────────────────────┘                    │
│                    ▼                                           │
│            ┌──────────────┐                                    │
│            │   AUGMENT    │  Query + Retrieved Docs            │
│            │   (Combine)  │  = Augmented Prompt                │
│            └──────┬───────┘                                    │
│                   │                                            │
│                   ▼                                            │
│            ┌──────────────┐                                    │
│            │   GENERATE   │                                    │
│            │    (LLM)     │                                    │
│            └──────┬───────┘                                    │
│                   │                                            │
│                   ▼                                            │
│              Answer grounded                                   │
│              in real documents                                 │
│                                                                │
└─────────────────────────────────────────────────────────────────┘
```

### 🔑 Key Analogy

| Standard LLM | RAG |
|--------------|-----|
| Closed-book exam | Open-book exam |
| Must answer from memory | Can look up information |
| May guess incorrectly | Answers grounded in sources |

---

## Part 3: The Retrieval Component

### 📝 What is Retrieval?

**Retrieval** = Finding relevant documents for a given query

This is exactly what you learned in **Weeks 2-5**! The process is:

```
Step 1: Convert all documents into vectors (numbers)
        ↓
Step 2: Convert the user's query into a vector
        ↓
Step 3: Calculate similarity between query and all documents
        ↓
Step 4: Return the top-k most similar documents
```

### 📝 What is "top-k"?

**k** is simply the number of documents to retrieve:
- **k=3** means retrieve the 3 most similar documents
- **k=5** means retrieve the 5 most similar documents

**Trade-off:** Higher k = more context, but may include irrelevant documents

### 📝 Dense vs Sparse Embeddings

| Type | Method | What it looks like | Pros | Cons |
|------|--------|-------------------|------|------|
| **Sparse** | TF-IDF (Week 2-3) | `[0, 0, 0, 1, 0, 0, ..., 0]` | Fast, interpretable | Only matches exact words |
| **Dense** | Neural embeddings | `[0.2, -0.5, 0.8, ..., 0.1]` | Captures meaning | Requires more computation |

### 🔑 Why Dense Embeddings are Better for RAG

**Example:** User asks "How many holiday days do I get?"

| Method | Can it find "Annual leave is 30 days"? | Why? |
|--------|---------------------------------------|------|
| TF-IDF (Sparse) | ❌ Low score | "holiday" ≠ "annual leave" (different words) |
| Dense Embeddings | ✅ High score | Understands "holiday" ≈ "annual leave" (same meaning) |

### 📝 What is Sentence-Transformers?

**Sentence-Transformers** is a library that provides pre-trained models to convert text into dense vectors.

We'll use `all-MiniLM-L6-v2` which:
- ✅ Is **free** and runs locally (no API key!)
- ✅ Produces **384-dimensional** vectors
- ✅ Is **fast** and works well for most tasks
- ✅ Understands **semantic similarity**

In [ ]:
# 💻 STEP 3: Load the embedding model
# First run downloads ~90MB - be patient!
print("📥 Loading embedding model (first run downloads ~90MB)...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Model loaded!")
print(f"   This model produces {embedding_model.get_sentence_embedding_dimension()}-dimensional vectors")

**Expected output:**
```
📥 Loading embedding model (first run downloads ~90MB)...
✅ Model loaded!
   This model produces 384-dimensional vectors
```

### 📝 Step 3.1: Create a Knowledge Base

In a real application, you would load documents from files. For this tutorial, we'll create a simple knowledge base of company policies.

In [ ]:
# 💻 STEP 4: Create our knowledge base
# These are example company policy documents
documents = [
    "Annual leave allowance is 30 days per year for all full-time employees.",
    "Sick leave requires a doctor's note after 3 consecutive days of absence.",
    "Remote working is permitted 2 days per week with manager approval.",
    "The office dress code is business casual from Monday to Thursday.",
    "Friday is casual dress day and employees may wear jeans.",
    "All employees must complete mandatory health and safety training annually.",
    "Parental leave is 26 weeks for primary carers and 4 weeks for secondary carers.",
    "The company provides a pension contribution of 5% of salary.",
    "Performance reviews are conducted twice per year in March and September.",
    "Expenses must be submitted within 30 days of being incurred."
]

print(f"✅ Knowledge base created with {len(documents)} documents")
print(f"\n📄 Example document: '{documents[0]}'")

### 📝 Step 3.2: Convert Documents to Embeddings

Now we convert each document into a vector. This is called **indexing**.

**What happens:**
```
"Annual leave is 30 days..."  →  [0.023, -0.156, 0.089, ..., 0.045]
                                        (384 numbers)
```

In [ ]:
# 💻 STEP 5: Create embeddings for all documents
# This is the INDEXING step - done ONCE before any queries
print("🔄 Creating document embeddings...")
doc_embeddings = embedding_model.encode(documents)

print(f"✅ Created embeddings!")
print(f"   Shape: {doc_embeddings.shape}")
print(f"   Meaning: {doc_embeddings.shape[0]} documents × {doc_embeddings.shape[1]} dimensions")
print(f"\n📊 First 5 values of document 1's embedding:")
print(f"   {doc_embeddings[0][:5]}")

**Expected output:**
```
🔄 Creating document embeddings...
✅ Created embeddings!
   Shape: (10, 384)
   Meaning: 10 documents × 384 dimensions

📊 First 5 values of document 1's embedding:
   [ 0.01234  -0.05678  0.12345  -0.09876  0.03456]
```
(Your exact numbers will be slightly different)

### 📝 Step 3.3: Create the Retrieval Function

Now we create a function that:
1. Takes a query
2. Converts it to an embedding
3. Finds the most similar documents

**This uses cosine similarity from Week 4!**

In [ ]:
# 💻 STEP 6: Define the retrieval function
def retrieve_documents(query, doc_embeddings, documents, embedding_model, top_k=3):
    """
    Retrieve the most relevant documents for a query.

    Parameters:
    -----------
    query : str
        The user's question
    doc_embeddings : numpy array
        Pre-computed embeddings for all documents
    documents : list
        List of original document texts
    embedding_model : SentenceTransformer
        The model used to create embeddings
    top_k : int
        Number of documents to retrieve (default: 3)

    Returns:
    --------
    list of tuples: [(document_text, similarity_score), ...]
    """
    # Step A: Convert query to embedding (MUST use same model!)
    query_embedding = embedding_model.encode([query])

    # Step B: Calculate cosine similarity with ALL documents
    similarities = cosine_similarity(query_embedding, doc_embeddings)[0]

    # Step C: Get indices of top-k most similar documents
    top_indices = np.argsort(similarities)[::-1][:top_k]

    # Step D: Return documents with their scores
    results = [(documents[i], similarities[i]) for i in top_indices]

    return results

print("✅ Retrieval function defined!")

### 💻 Let's Test the Retrieval!

In [ ]:
# 💻 STEP 7: Test the retrieval function
query = "How many holiday days do I get?"

print(f"🔍 Query: '{query}'")
print("\n" + "="*65)
print("📄 Retrieved documents (ranked by similarity):")
print("="*65)

results = retrieve_documents(query, doc_embeddings, documents, embedding_model, top_k=3)

for i, (doc, score) in enumerate(results, 1):
    print(f"\n[{i}] Similarity: {score:.3f}")
    print(f"    📄 {doc}")

### 🔑 Key Observation!

The query asked about **"holiday days"** but the top result mentions **"annual leave"**.

**This is the power of dense embeddings!** They understand that:
- "holiday" ≈ "annual leave"
- "days" ≈ "days"

TF-IDF would have given a LOW score because the exact words don't match!

### ✅ Exercise 3.1: Test Semantic Matching

Run the cell below and observe how the system finds relevant documents even when you use different words.

In [ ]:
# ✅ EXERCISE: Observe semantic matching in action
test_queries = [
    "What should I wear to the office?",      # Should match → dress code
    "Can I work from home?",                   # Should match → remote working
    "When is my performance evaluation?",      # Should match → performance reviews
    "What happens if I'm sick?"                # Should match → sick leave
]

print("Testing semantic matching:")
print("="*65)

for query in test_queries:
    results = retrieve_documents(query, doc_embeddings, documents, embedding_model, top_k=1)
    doc, score = results[0]
    print(f"\n🔍 Query: '{query}'")
    print(f"   → Best match [{score:.3f}]: {doc[:50]}...")

### ✅ Exercise 3.2: Compare Sparse vs Dense

Let's see how TF-IDF (sparse) compares to dense embeddings.

In [ ]:
# ✅ EXERCISE: Compare TF-IDF vs Dense Embeddings
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF vectors (sparse - like Week 2-3)
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(documents)

# Query where words DON'T match exactly
query = "How many holiday days do I get?"

# TF-IDF retrieval
query_tfidf = tfidf.transform([query])
tfidf_scores = cosine_similarity(query_tfidf, tfidf_matrix)[0]
tfidf_top = np.argmax(tfidf_scores)

# Dense retrieval
dense_results = retrieve_documents(query, doc_embeddings, documents, embedding_model, top_k=1)

print(f"🔍 Query: '{query}'")
print("\n" + "="*65)
print("TF-IDF (Sparse) - matches exact words only:")
print(f"   Score: {tfidf_scores[tfidf_top]:.3f}")
print(f"   Doc: {documents[tfidf_top][:50]}...")
print("\n" + "="*65)
print("Dense Embeddings - understands meaning:")
print(f"   Score: {dense_results[0][1]:.3f}")
print(f"   Doc: {dense_results[0][0][:50]}...")
print("\n🔑 Notice: Dense embeddings give a HIGHER score because they")
print("   understand that 'holiday' ≈ 'annual leave'!")

---

## Part 4: The Generation Component

### 📝 What is Generation?

After retrieving relevant documents, we need to **generate an answer** using an LLM.

**The key idea:** We don't just send the query to the LLM. We send the query **PLUS** the retrieved documents.

```
Standard LLM:     Query ──────────────────────────► LLM ──► Answer (may hallucinate)

RAG:              Query + Retrieved Documents ──► LLM ──► Grounded Answer ✓
```

### 📝 What is Prompt Augmentation?

**Augmentation** = Adding retrieved documents to the prompt

We create a prompt that says:
1. "Here is some context" (the retrieved documents)
2. "Here is the question"
3. "Answer based ONLY on the context" (reduces hallucinations)

In [ ]:
# 💻 STEP 8: Create the prompt augmentation function
def create_rag_prompt(query, retrieved_docs):
    """
    Create a prompt that includes retrieved context.
    This is the AUGMENTATION step.
    """
    # Format the retrieved documents
    context_parts = []
    for i, (doc, score) in enumerate(retrieved_docs, 1):
        context_parts.append(f"[Document {i}] {doc}")

    context = "\n".join(context_parts)

    # Create the prompt with clear instructions
    prompt = f"""Answer the question based ONLY on the following context.
If the context doesn't contain enough information, say "I don't have enough information."

Context:
{context}

Question: {query}

Answer:"""

    return prompt

print("✅ Prompt augmentation function defined!")

In [ ]:
# 💻 Let's see what an augmented prompt looks like
query = "How many holiday days do I get?"
retrieved = retrieve_documents(query, doc_embeddings, documents, embedding_model, top_k=2)

prompt = create_rag_prompt(query, retrieved)

print("📝 Generated RAG Prompt:")
print("="*65)
print(prompt)
print("="*65)

### 📝 Loading the Language Model

We'll use **FLAN-T5**, a free, open-source model from Google:
- ✅ Runs locally on Colab (no API key needed)
- ✅ Good at following instructions
- ✅ Small enough to run without a GPU

**⏱️ Note:** First run downloads ~1GB. This takes 2-3 minutes.

In [ ]:
# 💻 STEP 9: Load the language model
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

print("📥 Loading language model (first run downloads ~1GB)...")
print("⏱️  This may take 2-3 minutes...")

model_name = "google/flan-t5-base"  # Free, no API key needed!
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

generator = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100
)

print("✅ Language model loaded!")

### 📝 The Complete RAG Pipeline

Now let's put it all together:

```
User Query
    │
    ▼
┌─────────────────┐
│  1. RETRIEVE    │  ← Find relevant documents
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│  2. AUGMENT     │  ← Add documents to prompt
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│  3. GENERATE    │  ← LLM produces answer
└────────┬────────┘
         │
         ▼
   Grounded Answer
```

In [ ]:
# 💻 STEP 10: Define the complete RAG pipeline
def rag_answer(query, doc_embeddings, documents, embedding_model, generator, top_k=3):
    """
    Complete RAG pipeline: Retrieve → Augment → Generate
    """
    # Step 1: RETRIEVE relevant documents
    retrieved = retrieve_documents(query, doc_embeddings, documents, embedding_model, top_k)

    # Step 2: AUGMENT - create prompt with context
    prompt = create_rag_prompt(query, retrieved)

    # Step 3: GENERATE answer using the LLM
    response = generator(prompt)[0]['generated_text']

    return response, retrieved

print("✅ Complete RAG pipeline defined!")

In [ ]:
# 💻 STEP 11: Test the complete RAG system!
query = "How many holiday days do I get?"

print(f"🔍 Question: {query}")
print("\n🔄 Processing...")

answer, sources = rag_answer(query, doc_embeddings, documents, embedding_model, generator)

print("\n" + "="*65)
print(f"💬 Answer: {answer}")
print("="*65)
print("\n📚 Sources used:")
for doc, score in sources:
    print(f"   [{score:.3f}] {doc}")

### 🔑 Key Observation

The answer **"30 days"** comes directly from the retrieved document!

The LLM didn't hallucinate - it extracted the information from the context we provided.

### ✅ Exercise 4.1: Test with Different Questions

In [ ]:
# ✅ EXERCISE: Test the RAG system with multiple questions
test_questions = [
    "What is the dress code on Friday?",
    "How long is parental leave for primary carers?",
    "When do I need a doctor's note?",
    "Can I work from home?"
]

print("Testing RAG system:")
print("="*65)

for q in test_questions:
    answer, _ = rag_answer(q, doc_embeddings, documents, embedding_model, generator)
    print(f"\n❓ Q: {q}")
    print(f"💬 A: {answer}")

---

## Part 5: Chunking - Splitting Long Documents

### 📝 What is Chunking?

**Chunking** = Splitting long documents into smaller pieces

### 📝 Why Do We Need Chunking?

| Reason | Explanation |
|--------|-------------|
| **Embedding model limits** | Models can only process ~512 tokens at once |
| **LLM context limits** | Can't fit entire documents in the prompt |
| **Precision** | We want specific paragraphs, not whole documents |

### 📝 Chunk Size Trade-offs

```
Chunks TOO SMALL                    Chunks TOO LARGE
────────────────                    ────────────────
❌ Lose context                      ❌ Include irrelevant info
❌ May split sentences               ❌ Less precise retrieval
❌ "Annual leave is" / "30 days"    ❌ Whole handbook in one chunk
```

**🔑 Goal:** Each chunk should contain ONE complete, coherent piece of information.

### 📝 What is Overlap?

**Overlap** = Shared text between adjacent chunks

```
Without overlap:  [Chunk 1: "Annual leave is"] [Chunk 2: "30 days per year"]
                                           ↑
                                  Information split! ❌

With overlap:     [Chunk 1: "Annual leave is 30 days"]
                                       [Chunk 2: "is 30 days per year"]
                                           ↑
                                  Information preserved! ✓
```

**Rule of thumb:** Set overlap to 10-20% of chunk size

In [ ]:
# 💻 STEP 12: Chunking example
from langchain_text_splitters import RecursiveCharacterTextSplitter

# A longer document to demonstrate chunking
long_document = """
EMPLOYEE HANDBOOK - LEAVE POLICIES

Section 1: Annual Leave
All full-time employees are entitled to 30 days of annual leave per year.
Leave must be requested at least 2 weeks in advance for periods longer than 3 days.
Up to 5 days of unused leave may be carried over to the following year.

Section 2: Sick Leave
Employees may take sick leave when unwell without a doctor's note for up to 3 days.
For absences longer than 3 days, a medical certificate is required.
The company provides full pay for the first 10 days of sick leave per year.

Section 3: Parental Leave
Primary carers are entitled to 26 weeks of paid parental leave.
Secondary carers are entitled to 4 weeks of paid parental leave.
Parental leave must be taken within the first year of the child's birth.
"""

print(f"📄 Original document length: {len(long_document)} characters")

In [ ]:
# 💻 STEP 13: Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,      # Maximum characters per chunk
    chunk_overlap=50,    # Overlap between chunks
    separators=["\n\n", "\n", ". ", " "]  # Try these in order
)

chunks = text_splitter.split_text(long_document)

print(f"✅ Split into {len(chunks)} chunks\n")
print("="*65)
for i, chunk in enumerate(chunks, 1):
    print(f"\n📄 Chunk {i} ({len(chunk)} chars):")
    print("-"*40)
    print(chunk.strip())

### 📝 How RecursiveCharacterTextSplitter Works

It tries separators **in order**:

1. First: `\n\n` (paragraph breaks) - keeps paragraphs together
2. Then: `\n` (line breaks) - keeps lines together  
3. Then: `. ` (sentences) - keeps sentences together
4. Finally: ` ` (spaces) - splits words as last resort

This preserves natural text structure!

### ✅ Exercise 5.1: Experiment with Chunk Sizes

In [ ]:
# ✅ EXERCISE: Try different chunk sizes
chunk_sizes = [100, 200, 300, 500]

print("Comparing chunk sizes:")
print("="*50)

for size in chunk_sizes:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=size,
        chunk_overlap=size // 10  # 10% overlap
    )
    result_chunks = splitter.split_text(long_document)
    avg_len = np.mean([len(c) for c in result_chunks])

    print(f"\nChunk size {size}:")
    print(f"   → {len(result_chunks)} chunks (avg {avg_len:.0f} chars each)")

---

## Part 6: Vector Databases

### 📝 What is a Vector Database?

A **vector database** is designed to store and search embeddings efficiently.

| Regular Database | Vector Database |
|-----------------|----------------|
| Stores text, numbers | Stores vectors (embeddings) |
| Exact matching | Similarity matching |
| SQL queries | Vector similarity search |

### 📝 Why Use a Vector Database?

- When you have **thousands/millions** of documents
- Comparing every vector is **too slow**
- Vector DBs use special algorithms for **fast search**
- They **persist data** between sessions

In [ ]:
# 💻 STEP 14: Create a Chroma vector database
import chromadb
from chromadb.utils import embedding_functions

# Create a client
chroma_client = chromadb.Client()

# Create embedding function (same model as before!)
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

# Create a collection
collection = chroma_client.create_collection(
    name="employee_handbook",
    embedding_function=embedding_func
)

print("✅ Chroma vector database created!")

In [ ]:
# 💻 STEP 15: Add chunks to the database
collection.add(
    documents=chunks,
    ids=[f"chunk_{i}" for i in range(len(chunks))],
    metadatas=[{"source": "handbook", "chunk_id": i} for i in range(len(chunks))]
)

print(f"✅ Added {collection.count()} chunks to the database")

In [ ]:
# 💻 STEP 16: Query the vector database
query = "How many days can I carry over?"

results = collection.query(
    query_texts=[query],
    n_results=2
)

print(f"🔍 Query: '{query}'")
print("\n" + "="*65)
print("📄 Retrieved chunks:")
print("="*65)

for i, (doc, dist) in enumerate(zip(results['documents'][0], results['distances'][0]), 1):
    similarity = 1 - dist  # Convert distance to similarity
    print(f"\n[{i}] Similarity: {similarity:.3f}")
    print(f"    {doc[:100]}..." if len(doc) > 100 else f"    {doc}")

---

## Part 7: Evaluation

### 📝 Two Things to Evaluate

| Component | Question | If it fails... |
|-----------|----------|---------------|
| **Retrieval** | Did we find the right documents? | Wrong context → Wrong answer |
| **Generation** | Is the answer correct? | Even right context can give wrong answer |

### 🔑 Important: Evaluate Retrieval FIRST!

If retrieval fails, generation cannot succeed.

### 📝 Retrieval Metrics (from Week 5!)

| Metric | Formula | Meaning |
|--------|---------|--------|
| **Precision@k** | Relevant in top-k ÷ k | What % of retrieved docs are relevant? |
| **Recall@k** | Relevant in top-k ÷ Total relevant | What % of relevant docs did we find? |

In [ ]:
# 💻 STEP 17: Evaluation function
def evaluate_retrieval(retrieved_ids, relevant_ids):
    """
    Calculate Precision and Recall for retrieval.
    """
    retrieved = set(retrieved_ids)
    relevant = set(relevant_ids)

    true_positives = len(retrieved & relevant)

    precision = true_positives / len(retrieved) if retrieved else 0
    recall = true_positives / len(relevant) if relevant else 0

    return {'precision': precision, 'recall': recall}

# Example evaluation
query = "What is the parental leave policy?"

# Find which chunks are actually relevant (ground truth)
relevant_ids = [i for i, chunk in enumerate(chunks) if 'parental' in chunk.lower()]

# Get retrieved chunks
results = collection.query(query_texts=[query], n_results=3)
retrieved_ids = [int(id.split('_')[1]) for id in results['ids'][0]]

print(f"🔍 Query: '{query}'")
print(f"\n📊 Relevant chunk IDs: {relevant_ids}")
print(f"📊 Retrieved chunk IDs: {retrieved_ids}")

metrics = evaluate_retrieval(retrieved_ids, relevant_ids)
print(f"\n✅ Precision: {metrics['precision']:.2f}")
print(f"✅ Recall: {metrics['recall']:.2f}")

---

## Part 8: Common Failure Modes

### 📝 Understanding How RAG Can Fail

In [ ]:
# 💻 Failure Mode 1: Information not in knowledge base
query = "What is the company's policy on pets in the office?"

results = collection.query(query_texts=[query], n_results=2)

print(f"🔍 Query: '{query}'")
print("\n⚠️ Retrieved documents:")
for doc in results['documents'][0]:
    print(f"   → {doc[:60]}...")

print("\n❌ PROBLEM: The knowledge base has NO information about pets!")
print("   A good RAG system should say 'I don't have that information.'")

In [ ]:
# 💻 Failure Mode 2: Different terminology
query = "What's the PTO policy?"  # PTO = American term

results = collection.query(query_texts=[query], n_results=2)

print(f"🔍 Query: '{query}'")
print("\n📄 Retrieved:")
for doc, dist in zip(results['documents'][0], results['distances'][0]):
    print(f"   [{1-dist:.3f}] {doc[:50]}...")

print("\n⚠️ NOTE: Our documents use 'annual leave' (British), not 'PTO' (American).")
print("   Dense embeddings may still work, but it's not guaranteed.")

---

## Part 9: Summary

### 🔑 Key Takeaways

| Concept | What You Learned |
|---------|------------------|
| **RAG** | Retrieve → Augment → Generate |
| **Why RAG** | Fixes LLM limitations (cutoff, hallucinations, no private data) |
| **Dense embeddings** | Capture semantic meaning ("holiday" ≈ "annual leave") |
| **Chunking** | Split documents for precise retrieval |
| **Vector databases** | Fast similarity search at scale |
| **Evaluation** | Check retrieval AND generation separately |

### 📊 Recommended Settings

| Parameter | Starting Value | Notes |
|-----------|---------------|-------|
| Chunk size | 500-1000 chars | Adjust for your documents |
| Chunk overlap | 50-100 chars | ~10% of chunk size |
| Top-k | 3-5 | Balance coverage vs noise |
| Embedding model | all-MiniLM-L6-v2 | Free, fast, good quality |

---

## ✅ Lab Exercise: Build Your Own RAG System

**Task:** Build a RAG system for product reviews.

**Steps:**
1. Create a Chroma collection
2. Add the reviews
3. Query the system
4. Generate answers

In [ ]:
# Sample reviews for the exercise
sample_reviews = [
    "The pen writes smoothly and the ink quality is excellent. Great value.",
    "Disappointed. The scissors broke after just one use.",
    "These markers are perfect for art projects. Vibrant colours!",
    "Overpriced. I found better alternatives at half the price.",
    "The notebook paper is too thin. Ink bleeds through.",
    "Love these coloured pencils! They blend beautifully.",
    "Poor quality control. Two pens in the pack were dried out.",
    "Fast delivery and excellent packaging. Exceeded expectations.",
    "Not worth the money. The highlighters stopped working quickly.",
    "Best art supplies I've ever purchased. Will buy again!"
]

print(f"✅ Loaded {len(sample_reviews)} sample reviews")

In [ ]:
# ✅ YOUR CODE HERE - Try it yourself first!
# Step 1: Create a new collection called "reviews"
# review_collection = chroma_client.create_collection(...)

# Step 2: Add the reviews
# review_collection.add(...)

# Step 3: Query for "quality issues"
# results = review_collection.query(...)

# Step 4: Print the results

print("💻 Try implementing this yourself first, then check the solution below!")

### 📝 Solution

Once you've tried it yourself, check your solution against this:

In [ ]:
# ✅ SOLUTION - Step 1: Create a new collection
review_collection = chroma_client.create_collection(
    name="product_reviews",
    embedding_function=embedding_func
)
print("✅ Step 1: Collection created!")

In [ ]:
# ✅ SOLUTION - Step 2: Add the reviews to the collection
review_collection.add(
    documents=sample_reviews,
    ids=[f"review_{i}" for i in range(len(sample_reviews))],
    metadatas=[{"source": "art_supplies", "review_id": i} for i in range(len(sample_reviews))]
)
print(f"✅ Step 2: Added {review_collection.count()} reviews to the collection!")

In [ ]:
# ✅ SOLUTION - Step 3: Query the collection
test_queries = [
    "Find reviews about quality issues",
    "What do people say about the price?",
    "Are there positive reviews about pens?"
]

print("✅ Step 3: Querying the review collection")
print("=" * 65)

for query in test_queries:
    results = review_collection.query(
        query_texts=[query],
        n_results=3
    )

    print(f"\n🔍 Query: '{query}'")
    print("-" * 50)
    for i, (doc, dist) in enumerate(zip(results['documents'][0], results['distances'][0]), 1):
        similarity = 1 - dist
        print(f"   [{i}] ({similarity:.3f}) {doc}")

In [ ]:
# ✅ SOLUTION - Step 4: Use the full RAG pipeline to generate answers
# First, create embeddings for the reviews (for our rag_answer function)
review_embeddings = embedding_model.encode(sample_reviews)

print("✅ Step 4: Full RAG pipeline with reviews")
print("=" * 65)

rag_queries = [
    "What are the main quality complaints?",
    "Which products are recommended?"
]

for query in rag_queries:
    answer, sources = rag_answer(query, review_embeddings, sample_reviews, embedding_model, generator, top_k=3)

    print(f"\n❓ Question: {query}")
    print(f"💬 Answer: {answer}")
    print("📚 Sources:")
    for doc, score in sources:
        print(f"   [{score:.3f}] {doc[:50]}...")

### 🔑 What You Learned in This Lab

1. **Creating a collection** - `chroma_client.create_collection()`
2. **Adding documents** - `collection.add(documents=..., ids=..., metadatas=...)`
3. **Querying** - `collection.query(query_texts=[...], n_results=k)`
4. **Full RAG pipeline** - Combining retrieval with generation

### ✅ Self-Check Questions

Can you answer these?

1. Why do we need unique IDs for each document?
2. What happens if you increase `n_results` to 5?
3. What's the difference between the Chroma query and our `retrieve_documents` function?
4. How would you add new reviews to an existing collection?

---

## 📚 References

**Original RAG Paper:**
- Lewis et al. (2020), "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks"
- https://arxiv.org/abs/2005.11401

**Tools:**
- Sentence-Transformers: https://www.sbert.net/
- Chroma: https://www.trychroma.com/
- LangChain: https://python.langchain.com/

**Further Reading:**
- "Lost in the Middle" (Liu et al., 2023)
- RAGAS evaluation framework